### Automatic model selection

Test notebook for automatic model selection.

In [1]:
import datetime
import logging
import sys
import numpy as np
import time

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from keras.callbacks import LearningRateScheduler
from keras.optimizers import Adam
import keras.backend as K

#sys.path.append('/Users/davidlaredorazo/Documents/University_of_California/Research/Projects')
sys.path.append('/media/controlslab/DATA/Projects')

import ann_framework.aux_functions as aux_functions

import automatic_model_selection
from automatic_model_selection import Configuration
from ann_encoding_rules import Layers
import fetch_to_keras
#from CMAPSAuxFunctions import TrainValTensorBoard

#Tunable model
from ann_framework.tunable_model.tunable_model import SequenceTunableModelRegression, SequenceTunableModelClassification

#Data handlers
from ann_framework.data_handlers.data_handler_CMAPSS import CMAPSSDataHandler
from ann_framework.data_handlers.data_handler_MNIST import MNISTDataHandler
from ann_framework.data_handlers.data_handler_CIFAR10 import CIFAR10DataHandler

learningRate_scheduler = LearningRateScheduler(aux_functions.step_decay)

size_scaler = 1

#Use same configuration for all experiments, just change some of the parameters

#Define some random paramaters for the creation of the configuration, this will change for each test model
architecture_type = Layers.FullyConnected
problem_type = 2  #1 for regression, 2 for classification
output_shape = 10 #If regression applies, number of classes
input_shape = (784,)

config = Configuration(architecture_type, problem_type, input_shape, output_shape, pop_size=5, 
                       tournament_size=3, max_similar=3, epochs=20, cross_val=0.2, size_scaler=size_scaler,
                       max_generations=10, binary_selection=True, mutation_ratio=0.4, 
                       similarity_threshold=0.2, more_layers_prob=0.7, verbose_individuals=True, 
                       show_model=True, verbose_training=1)


Using TensorFlow backend.


### Given a model, get the compiled model

In [2]:
def get_compiled_model(model, problem_type, optimizer_params=[]):
    """Obtain a keras compiled model"""
    
    #Shared parameters for the models
    optimizer = Adam(lr=0.001, beta_1=0.5)
    
    if problem_type == 1:
        lossFunction = "mean_squared_error"
        metrics = ["mse"]
    elif problem_type == 2:
        lossFunction = "categorical_crossentropy"
        metrics = ["accuracy"]
    else:
        print("Problem type not defined")
        model = None
        return
    
    #Create and compile the models
    model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    
    return model


def create_tunable_model(model_genotype, problem_type, input_shape, data_handler, model_number):
    
    K.clear_session()
    
    model = fetch_to_keras.decode_genotype(model_genotype, problem_type, input_shape, 1)
    
    model = get_compiled_model(model, problem_type, optimizer_params=[])
    
    if problem_type == 1:
        tModel = SequenceTunableModelRegression('ModelReg_SN_'+str(model_number), model, lib_type='keras', data_handler=data_handler)
    else:
        tModel = SequenceTunableModelClassification('ModelClass_SN_'+str(model_number), model, lib_type='keras', data_handler=data_handler)
        
    return tModel

### Load cmaps data handler

In [3]:
def cmaps_dhandler():

    #Selected as per CNN paper
    features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 'BPR', 
    'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
    selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
    selected_features = list(features[i] for i in selected_indices-1)
    data_folder = '../CMAPSSData'

    window_size = 24
    window_stride = 1
    max_rul = 129

    dHandler_cmaps = CMAPSSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)

    input_shape = (len(selected_features)*window_size, )

    return dHandler_cmaps, input_shape

### Function to save top models

In [4]:
def save_best_models(best_models_list, global_best_model_index, saveto, input_shape, data_handler, 
                     problem_type=1, data_scaler=None, train_epochs=100, metrics=[], round=0):
    
    n_models = len(best_models_list)
    
    for ind_model, i in zip(best_models_list, range(n_models)):
        
        tModel = create_tunable_model(ind_model.stringModel, problem_type, input_shape, data_handler, i)
        kmodel = tModel.model
        model_json = kmodel.to_json()
        
        #Save model's architecture
        string_append = str(i) if i != global_best_model_index else 'global'
        with open(saveto+"bestModel_"+string_append+".json", "w") as json_file:
            json_file.write(model_json)
            
    #Train the global best, model has to be recompiled
    ind_model = best_models_list[global_best_model_index]
    tModel = create_tunable_model(ind_model.stringModel, problem_type, input_shape, data_handler, n_models)
    
    print(tModel.model.summary())
    print(tModel.data_handler)
    
    if tModel.data_handler.data_scaler != None:
        print("Using data handler scaler")
    elif tModel.data_scaler != None:
        print("Using tModel scaler (Overriding data handler scaler)")
    else:
        print("No data scaling used")
    
    if data_scaler != None:
        tModel.data_handler.data_scaler = None
        tModel.data_scaler = data_scaler
        
    tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
    tModel.print_data()
    tModel.epochs = train_epochs

    tModel.train_model(verbose=1)
    
    tModel.evaluate_model(metrics, round=round)
    
    kmodel = tModel.model
            
    # serialize weights to HDF5
    kmodel.save_weights(saveto+"bestModel_global.h5")
    
    print("Saved models for dataset 1 to disk")
        

### Test on MNIST

In [ ]:
"""Input can be of 3 types, ANN (1), CNN (2) or RNN (3)"""
architecture_type = Layers.FullyConnected
problem_type = 2  #1 for regression, 2 for classification
output_shape = 10 #If regression applies, number of classes
input_shape = (784,)

"""
pop_size = 5
tournament_size = 3
max_similar = 3
total_experiments = 5
count_experiments = 0
unroll = True
"""
total_experiments = 5
count_experiments = 0
unroll = True

global_best_list = []
global_best = None
global_best_index = 0
experiment_times = np.zeros((total_experiments,1))

scaler = None

t = datetime.datetime.now()

logging.basicConfig(filename='logs/nn_evolution_mnist_' + t.strftime('%m%d%Y%H%M%S') + '.log', level=logging.INFO, 
                        format='%(levelname)s:%(threadName)s:%(message)s', datefmt='%m/%d/%Y %H:%M:%S')

#mnist datahandler
dHandler_mnist = MNISTDataHandler()

config.architecture_type = architecture_type
config.problem_type = problem_type
config.input_shape = input_shape
config.output_shape = output_shape

"""
config = Configuration(architecture_type, problem_type, input_shape, output_shape, pop_size, 
                       tournament_size, max_similar, epochs=20, cross_val=0.2, size_scaler=size_scaler,
                       max_generations=10, binary_selection=True, mutation_ratio=0.4, 
                       similarity_threshold=0.2, more_layers_prob=0.8)
"""

while count_experiments < total_experiments:
    print("Launching experiment {}".format(count_experiments+1))
    logging.info("Launching experiment {}".format(count_experiments+1))
    
    
    start = time.time()
    best = automatic_model_selection.run_experiment(config, dHandler_mnist, count_experiments + 1, unroll=unroll,
                                                    learningRate_scheduler=learningRate_scheduler, 
                                                    tModel_scaler=scaler)
    end = time.time()
    elapsed_time = (end-start)/60
    experiment_times[count_experiments] = elapsed_time
    print("Experiment time: {} minutes".format(elapsed_time))
    logging.info("Experiment time: {} minutes".format(elapsed_time))
    

    best.individual_label = count_experiments
    
    global_best_list.append(best)

    if global_best == None:
        global_best = best
    else:
        if best.fitness < global_best.fitness:
            global_best = best
            global_best_index = count_experiments

    count_experiments =  count_experiments + 1

print("Global best list\n")
logging.info("Global best list\n")
automatic_model_selection.print_best(global_best_list)

print("Global best is\n")
print(global_best)
logging.info("Global best is\n")
logging.info(global_best)

print("Global time {}".format(experiment_times.sum()))
logging.info("Global time {}".format(experiment_times.sum()))

In [ ]:
save_best_models(global_best_list, global_best_index, 'best_models/mnist/alpha{}/epochs10/'.format(size_scaler), 
                 input_shape=input_shape, data_handler=dHandler_mnist, problem_type=problem_type, train_epochs=100)

### Test on CIFAR10

In [ ]:
"""Input can be of 3 types, ANN (1), CNN (2) or RNN (3)"""
architecture_type = Layers.FullyConnected
problem_type = 2  #1 for regression, 2 for classification
output_shape = 10 #If regression applies, number of classes
input_shape = (3072,)
"""
pop_size = 5
tournament_size = 3
max_similar = 3
"""
total_experiments = 5
count_experiments = 0
unroll = True

global_best_list = []
global_best = None
global_best_index = 0

scaler = None

t = datetime.datetime.now()

logging.basicConfig(filename='logs/nn_evolution_cifar10_' + t.strftime('%m%d%Y%H%M%S') + '.log', level=logging.INFO, 
                        format='%(levelname)s:%(threadName)s:%(message)s', datefmt='%m/%d/%Y %H:%M:%S')

#mnist datahandler
dHandler_cifar = CIFAR10DataHandler()

"""
config = Configuration(architecture_type, problem_type, input_shape, output_shape, pop_size, tournament_size, max_similar, 
                       epochs=5, cross_val=0.2, size_scaler=size_scaler, max_generations=10, binary_selection=True, 
                       mutation_ratio=0.4, similarity_threshold=0.2, more_layers_prob=0.8)
"""

config.architecture_type = architecture_type
config.problem_type = problem_type
config.input_shape = input_shape
config.output_shape = output_shape

while count_experiments < total_experiments:
    print("Launching experiment {}".format(count_experiments+1))
    logging.info("Launching experiment {}".format(count_experiments+1))
    
    best = automatic_model_selection.run_experiment(config, dHandler_cifar, count_experiments + 1, unroll=unroll,
                                                    learningRate_scheduler=learningRate_scheduler, 
                                                    tModel_scaler=scaler, verbose_data=0)
    
    best.individual_label = count_experiments

    global_best_list.append(best)

    if global_best == None:
        global_best = best
    else:
        if best.fitness < global_best.fitness:
            global_best = best
            global_best_index = count_experiments

    count_experiments =  count_experiments + 1

print("Global best list\n")
logging.info("Global best list\n")
automatic_model_selection.print_best(global_best_list)

print("Global best is\n")
print(global_best)
logging.info("Global best is\n")
logging.info(global_best)

In [ ]:
save_best_models(global_best_list, global_best_index, 'best_models/cifar10/alpha{}/'.format(size_scaler), 
                 input_shape=input_shape, data_handler=dHandler_cifar, problem_type=problem_type, train_epochs=100)

### Test on CMAPSS

In [5]:
"""Input can be of 3 types, ANN (1), CNN (2) or RNN (3)"""
architecture_type = Layers.FullyConnected
problem_type = 1  #1 for regression, 2 for classification
output_shape = 1 #If regression applies, number of classes
input_shape = None

"""
pop_size = 5
tournament_size = 3
max_similar = 3
"""
total_experiments = 5
count_experiments = 0
unroll = True

global_best_list = []
global_best = None
global_best_index = 0
experiment_times = np.zeros((total_experiments,1))

scaler = MinMaxScaler(feature_range=(-1, 1))

t = datetime.datetime.now()

logging.basicConfig(filename='logs/nn_evolution_cmaps_' + t.strftime('%m%d%Y%H%M%S') + '.log', level=logging.INFO, 
                        format='%(levelname)s:%(threadName)s:%(message)s', datefmt='%m/%d/%Y %H:%M:%S')

#cmaps datahandler
dhandler_cmaps, input_shape = cmaps_dhandler()

"""
config = Configuration(architecture_type, problem_type, input_shape, output_shape, pop_size, tournament_size, 
                       max_similar, epochs=5, cross_val=0.2, size_scaler=size_scaler, max_generations=10, 
                       binary_selection=True, mutation_ratio=0.4, similarity_threshold=0.2, more_layers_prob=0.8)
"""

config.architecture_type = architecture_type
config.problem_type = problem_type
config.input_shape = input_shape
config.output_shape = output_shape

while count_experiments < total_experiments:
    print("Launching experiment {}".format(count_experiments+1))
    logging.info("Launching experiment {}".format(count_experiments+1))
    
    start = time.time()
    best = automatic_model_selection.run_experiment(config, dhandler_cmaps, count_experiments + 1, unroll=unroll,
                                                    learningRate_scheduler=learningRate_scheduler, 
                                                    tModel_scaler=scaler)
    end = time.time()
    elapsed_time = (end-start)/60
    experiment_times[count_experiments] = elapsed_time
    print("Experiment time: {} minutes".format(elapsed_time))
    logging.info("Experiment time: {} minutes".format(elapsed_time))
    
    best.individual_label = count_experiments

    global_best_list.append(best)

    if global_best == None:
        global_best = best
    else:
        if best.fitness < global_best.fitness:
            global_best = best
            global_best_index = count_experiments

    count_experiments =  count_experiments + 1

print("Global best list\n")
logging.info("Global best list\n")
automatic_model_selection.print_best(global_best_list)

print("Global best is\n")
print(global_best)
logging.info("Global best is\n")
logging.info(global_best)

print("Global time {}".format(experiment_times.sum()))
logging.info("Global time {}".format(experiment_times.sum()))

Launching experiment 1

Generation 1
launch new
True
gen similar
False
Fetching model 0 to keras
Evaluating model 0
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 456)               153672    
_________________________________________________________________
dense_1 (Dense)              (None, 880)               402160    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 881       
Total params: 556,713
Trainable params: 556,713
Non-trainable params: 0
_________________________________________________________________
None
training with cv
Train on 14646 samples, validate on 20 samples
Epoch 1/20
14646/14646 [==============================] - 1s 45us/step - loss: 5800.6273 - mean_squared_error: 5800.6273 - val_loss: 5318.4131 - val_mean_squared_error: 5318.4131
Epoch 2/20
14646/14646 [============

14795/14795 [==============================] - 0s 5us/step - loss: 247.0856 - mean_squared_error: 247.0856 - val_loss: 1897.2776 - val_mean_squared_error: 1897.2776
Epoch 15/20
14795/14795 [==============================] - 0s 5us/step - loss: 271.0198 - mean_squared_error: 271.0198 - val_loss: 1906.7340 - val_mean_squared_error: 1906.7340
Epoch 16/20
14795/14795 [==============================] - 0s 5us/step - loss: 246.7098 - mean_squared_error: 246.7098 - val_loss: 1924.5641 - val_mean_squared_error: 1924.5641
Epoch 17/20
14795/14795 [==============================] - 0s 5us/step - loss: 246.5224 - mean_squared_error: 246.5224 - val_loss: 2251.2100 - val_mean_squared_error: 2251.2100
Epoch 18/20
14795/14795 [==============================] - 0s 5us/step - loss: 253.9948 - mean_squared_error: 253.9948 - val_loss: 1878.8422 - val_mean_squared_error: 1878.8422
Epoch 19/20
14795/14795 [==============================] - 0s 5us/step - loss: 240.3130 - mean_squared_error: 240.3130 - val_lo

15006/15006 [==============================] - 0s 4us/step - loss: 3062.4742 - mean_squared_error: 3062.4742 - val_loss: 4385.4575 - val_mean_squared_error: 4385.4575
Epoch 4/20
15006/15006 [==============================] - 0s 4us/step - loss: 2171.2411 - mean_squared_error: 2171.2411 - val_loss: 3539.5298 - val_mean_squared_error: 3539.5298
Epoch 5/20
15006/15006 [==============================] - 0s 4us/step - loss: 1595.9689 - mean_squared_error: 1595.9689 - val_loss: 2954.8542 - val_mean_squared_error: 2954.8542
Epoch 6/20
15006/15006 [==============================] - 0s 4us/step - loss: 1205.1651 - mean_squared_error: 1205.1651 - val_loss: 2509.3645 - val_mean_squared_error: 2509.3645
Epoch 7/20
15006/15006 [==============================] - 0s 4us/step - loss: 922.4158 - mean_squared_error: 922.4158 - val_loss: 2095.2788 - val_mean_squared_error: 2095.2788
Epoch 8/20
15006/15006 [==============================] - 0s 4us/step - loss: 722.6213 - mean_squared_error: 722.6213 - val

14249/14249 [==============================] - 0s 6us/step - loss: 6442.1175 - mean_squared_error: 6442.1175 - val_loss: 12895.2354 - val_mean_squared_error: 12895.2354
Epoch 17/20
14249/14249 [==============================] - 0s 6us/step - loss: 6353.8980 - mean_squared_error: 6353.8980 - val_loss: 12773.1191 - val_mean_squared_error: 12773.1191
Epoch 18/20
14249/14249 [==============================] - 0s 6us/step - loss: 6266.9463 - mean_squared_error: 6266.9463 - val_loss: 12652.3926 - val_mean_squared_error: 12652.3926
Epoch 19/20
14249/14249 [==============================] - 0s 7us/step - loss: 6181.2481 - mean_squared_error: 6181.2481 - val_loss: 12533.0283 - val_mean_squared_error: 12533.0283
Epoch 20/20
20/20 [==============================] - 0s 25us/step
Individual 4 score/size/fitness 12415.1171875/1146217/12421.176372117632

Generating offsprings
Applying Mutation
Launch new generation?: True

Generation 2
launch new
True
gen similar
False
Fetching model 0 to keras
Evalu

14440/14440 [==============================] - 0s 4us/step - loss: 763.9057 - mean_squared_error: 763.9057 - val_loss: 1868.5309 - val_mean_squared_error: 1868.5309
Epoch 10/20
14440/14440 [==============================] - 0s 4us/step - loss: 668.8670 - mean_squared_error: 668.8670 - val_loss: 1757.9592 - val_mean_squared_error: 1757.9592
Epoch 11/20
14440/14440 [==============================] - 0s 4us/step - loss: 555.4975 - mean_squared_error: 555.4975 - val_loss: 1594.5970 - val_mean_squared_error: 1594.5970
Epoch 12/20
14440/14440 [==============================] - 0s 4us/step - loss: 454.8628 - mean_squared_error: 454.8628 - val_loss: 1434.3749 - val_mean_squared_error: 1434.3749
Epoch 13/20
14440/14440 [==============================] - 0s 4us/step - loss: 383.8412 - mean_squared_error: 383.8412 - val_loss: 1310.4362 - val_mean_squared_error: 1310.4362
Epoch 14/20
14440/14440 [==============================] - 0s 4us/step - loss: 339.3410 - mean_squared_error: 339.3410 - val_lo

...

training with cv
Train on 14411 samples, validate on 20 samples
Epoch 1/20
14411/14411 [==============================] - 0s 17us/step - loss: 1830.2907 - mean_squared_error: 1830.2907 - val_loss: 1503.4705 - val_mean_squared_error: 1503.4705
Epoch 2/20
14411/14411 [==============================] - 0s 5us/step - loss: 347.5822 - mean_squared_error: 347.5822 - val_loss: 1476.1273 - val_mean_squared_error: 1476.1273
Epoch 3/20
14411/14411 [==============================] - 0s 5us/step - loss: 309.0178 - mean_squared_error: 309.0178 - val_loss: 2480.8132 - val_mean_squared_error: 2480.8132
Epoch 4/20
14411/14411 [==============================] - 0s 5us/step - loss: 295.9301 - mean_squared_error: 295.9301 - val_loss: 1596.3344 - val_mean_squared_error: 1596.3344
Epoch 5/20
14411/14411 [==============================] - 0s 5us/step - loss: 261.6562 - mean_squared_error: 261.6562 - val_loss: 1867.6481 - val_mean_squared_error: 1867.6481
Epoch 6/20
14411/14411 [=============================

Evaluating model 4
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 1008)              339696    
_________________________________________________________________
dense_1 (Dense)              (None, 448)               452032    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 449       
Total params: 792,177
Trainable params: 792,177
Non-trainable params: 0
_________________________________________________________________
None
training with cv
Train on 14545 samples, validate on 20 samples
Epoch 1/20
14545/14545 [==============================] - 0s 14us/step - loss: 4982.9152 - mean_squared_error: 4982.9152 - val_loss: 8046.3530 - val_mean_squared_error: 8046.3530
Epoch 2/20
14545/14545 [==============================] - 0s 4us/step - loss: 3241.0661 - mean_squared_error: 3241.0661 - val_loss: 6

14506/14506 [==============================] - 0s 3us/step - loss: 334.2599 - mean_squared_error: 334.2599 - val_loss: 2762.7090 - val_mean_squared_error: 2762.7090
Epoch 18/20
14506/14506 [==============================] - 0s 3us/step - loss: 312.5131 - mean_squared_error: 312.5131 - val_loss: 2714.6648 - val_mean_squared_error: 2714.6648
Epoch 19/20
14506/14506 [==============================] - 0s 3us/step - loss: 294.8691 - mean_squared_error: 294.8691 - val_loss: 2626.0566 - val_mean_squared_error: 2626.0566
Epoch 20/20
20/20 [==============================] - 0s 21us/step
Individual 0 score/size/fitness 2591.53857421875/151425/2596.717551166043
Fetching model 1 to keras
Evaluating model 1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 448)               150976    
_________________________________________________________________
dense_1 (Dense)              (No

14801/14801 [==============================] - 0s 4us/step - loss: 308.7422 - mean_squared_error: 308.7422 - val_loss: 1386.9377 - val_mean_squared_error: 1386.9377
Epoch 13/20
14801/14801 [==============================] - 0s 4us/step - loss: 287.1857 - mean_squared_error: 287.1857 - val_loss: 1320.8389 - val_mean_squared_error: 1320.8389
Epoch 14/20
14801/14801 [==============================] - 0s 4us/step - loss: 302.2848 - mean_squared_error: 302.2848 - val_loss: 1286.4503 - val_mean_squared_error: 1286.4503
Epoch 15/20
14801/14801 [==============================] - 0s 4us/step - loss: 263.5161 - mean_squared_error: 263.5161 - val_loss: 1219.3519 - val_mean_squared_error: 1219.3519
Epoch 16/20
14801/14801 [==============================] - 0s 4us/step - loss: 243.1021 - mean_squared_error: 243.1021 - val_loss: 1209.1078 - val_mean_squared_error: 1209.1078
Epoch 17/20
14801/14801 [==============================] - 0s 4us/step - loss: 240.4067 - mean_squared_error: 240.4067 - val_lo

15051/15051 [==============================] - 0s 6us/step - loss: 1266.7812 - mean_squared_error: 1266.7812 - val_loss: 1733.1049 - val_mean_squared_error: 1733.1049
Epoch 6/20
15051/15051 [==============================] - 0s 6us/step - loss: 1968.9453 - mean_squared_error: 1968.9453 - val_loss: 2380.6721 - val_mean_squared_error: 2380.6721
Epoch 7/20
15051/15051 [==============================] - 0s 6us/step - loss: 1147.0795 - mean_squared_error: 1147.0795 - val_loss: 1609.8041 - val_mean_squared_error: 1609.8041
Epoch 8/20
15051/15051 [==============================] - 0s 6us/step - loss: 922.8321 - mean_squared_error: 922.8321 - val_loss: 1492.1198 - val_mean_squared_error: 1492.1198
Epoch 9/20
15051/15051 [==============================] - 0s 6us/step - loss: 946.7987 - mean_squared_error: 946.7987 - val_loss: 1605.9495 - val_mean_squared_error: 1605.9495
Epoch 10/20
15051/15051 [==============================] - 0s 6us/step - loss: 736.0187 - mean_squared_error: 736.0187 - val_

In [6]:
save_best_models(global_best_list, global_best_index, 'best_models/cmapss/alpha{}/'.format(size_scaler), 
                 input_shape=input_shape, data_handler=dhandler_cmaps, problem_type=problem_type, train_epochs=100, 
                 data_scaler=scaler, metrics=['rhs', 'rmse'], round=2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 336)               113232    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 337       
Total params: 113,569
Trainable params: 113,569
Non-trainable params: 0
_________________________________________________________________
None
No data scaling used
Loading data for dataset 1 with window_size of 24, stride of 1 and maxRUL of 129. Cros-Validation ratio 0
Loading data from file and computing dataframes
Printing shapes

Training data (X, y)
(18331, 336)
(18331, 1)
Testing data (X, y)
(100, 336)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[-0.59060403 -0.0455243  -0.27982732 ... -0.45454545  0.33333333
   0.33501825]
 [-0.36912752  0.0629156  -0.18014129 ... -0.27272727  0.25581395
   0.50126369]
 [-0.23489933 -0.13299233 -0.13854003 ..

18331/18331 [==============================] - 0s 3us/step - loss: 213.1646 - mean_squared_error: 213.1646
Epoch 54/100
18331/18331 [==============================] - 0s 3us/step - loss: 212.2485 - mean_squared_error: 212.2485
Epoch 55/100
18331/18331 [==============================] - 0s 3us/step - loss: 211.7012 - mean_squared_error: 211.7012
Epoch 56/100
18331/18331 [==============================] - 0s 3us/step - loss: 212.0322 - mean_squared_error: 212.0322
Epoch 57/100
18331/18331 [==============================] - 0s 3us/step - loss: 211.0528 - mean_squared_error: 211.0528
Epoch 58/100
18331/18331 [==============================] - 0s 3us/step - loss: 208.8393 - mean_squared_error: 208.8393
Epoch 59/100
18331/18331 [==============================] - 0s 3us/step - loss: 208.6033 - mean_squared_error: 208.6033
Epoch 60/100
18331/18331 [==============================] - 0s 3us/step - loss: 207.4697 - mean_squared_error: 207.4697
Epoch 61/100
18331/18331 [===========================